In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import keras

from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization, Dropout, GaussianDropout, GaussianNoise, concatenate, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, Nadam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator 
from keras import backend as K
from keras import regularizers
from keras import losses
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from joblib import dump, load

In [ ]:
# Model Architecture

def fire_incept(layer: keras.layers = None, fire: int = None, intercept: int = None, names: list = None) -> keras.layers:    
    x = Conv2D(filters=fire, kernel_size=(1,1), name=names[0])(layer)
    x = LeakyReLU(alpha=0.15, name=names[1])(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding="same", name=names[2])(x)
    
    left = Conv2D(filters=intercept, kernel_size=(3,3), padding='same', name=names[3])(x)
    left = LeakyReLU(alpha=0.15, name=names[4])(left)
    
    right = Conv2D(filters=intercept, kernel_size=(5,5), padding='same', name=names[5])(x)
    right = LeakyReLU(alpha=0.15, name=names[6])(right)
    
    pool = Conv2D(filters=fire, kernel_size=(1,1), name=names[7])(layer)
    pool = LeakyReLU(alpha=0.15, name=names[8])(pool)
    
    x = concatenate([left, right, pool], axis=3, name=names[9])
    

    return x

def fire_squeeze(layer: keras.layers = None, fire: int = None, intercept: int = None, names: list = None) -> keras.layers:  
    x = Conv2D(filters=fire, kernel_size=(1,1), name=names[0])(layer)
    x = LeakyReLU(alpha=0.15, name=names[1])(x)
    
    left = Conv2D(filters=intercept, kernel_size=(1,1), name=names[2])(x)
    left = LeakyReLU(alpha=0.15, name=names[3])(left)
    
    right = Conv2D(filters=intercept, kernel_size=(3,3), padding='same', name=names[4])(x)
    right = LeakyReLU(alpha=0.15, name=names[5])(right)
    
    x = concatenate([left, right], axis=3, name=names[6])

    return x
  
image_input = Input(shape=(64, 64, 3), name="Input_1")

x = fire_incept(layer=image_input, fire=8, intercept=8, names=["Conv2D_1", 
                                                               "Leaky_ReLU_1",
                                                               "MaxPooling2D_1",
                                                               "Conv2D_2",
                                                               "Leaky_ReLU_2",
                                                               "Conv2D_3",
                                                               "Leaky_ReLU_3",
                                                               "Conv2D_4",
                                                               "Leaky_ReLU_4",
                                                               "Concatenate_1"])
x = BatchNormalization(name="Batch_Normalization_1")(x)

x = fire_incept(layer=x , fire=16, intercept=16, names=["Conv2D_5",
                                                        "Leaky_ReLU_5",
                                                        "MaxPooling2D_2",
                                                        "Conv2D_6",
                                                        "Leaky_ReLU_6",
                                                        "Conv2D_7",
                                                        "Leaky_ReLU_7",
                                                        "Conv2D_8",
                                                        "Leaky_ReLU_8",
                                                        "Concatenate_2"])

x = fire_squeeze(layer=x, fire=16, intercept=16, names=["Conv2D_9", 
                                                        "Leaky_ReLU_9",
                                                        "Conv2D_10", 
                                                        "Leaky_ReLU_10",
                                                        "Conv2D_11", 
                                                        "Leaky_ReLU_11",
                                                        "Concatenate_3"])
x = BatchNormalization(name="Batch_Normalization_2")(x)

x = fire_incept(layer=x, fire=16, intercept=16, names=["Conv2D_12", 
                                                       "Leaky_ReLU_12",
                                                       "Conv2D_13", 
                                                       "Leaky_ReLU_13",
                                                       "Conv2D_14", 
                                                       "Leaky_ReLU_14",
                                                       "MaxPooling2D_3",
                                                       "Conv2D_15", 
                                                       "Leaky_ReLU_15",
                                                       "Concatenate_4"])

x = fire_squeeze(layer=x, fire=16, intercept=16, names=["Conv2D_16", 
                                                        "Leaky_ReLU_16",
                                                        "Conv2D_17", 
                                                        "Leaky_ReLU_17",
                                                        "Conv2D_18", 
                                                        "Leaky_ReLU_18",
                                                        "Concatenate_5"])
x = BatchNormalization(name="Batch_Normalization_3")(x)

x = GlobalAveragePooling2D(name="GlobalAveragePooling2D_1")(x)

output = Dense(units=9, name="Dense_1")(x)

output = Activation("softmax", name="Activation_1")(output)

model = Model(inputs=image_input, outputs=output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
Conv2D_1 (Conv2D)               (None, 64, 64, 8)    32          Input_1[0][0]                    
__________________________________________________________________________________________________
Leaky_ReLU_1 (LeakyReLU)        (None, 64, 64, 8)    0           Conv2D_1[0][0]                   
__________________________________________________________________________________________________
MaxPooling2D_1 (MaxPooling2D)   (None, 64, 64, 8)    0           Leaky_ReLU_1[0][0]               
______________________________________________________________________________________________

In [ ]:
# Compiling Model
optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

model.compile(optimizer=optimizer, loss=losses.categorical_crossentropy, metrics=["accuracy"])

In [ ]:
# Fitting training and test images into the model
# history = model.fit(
#         x=x_train,
#         y=y_train,
#         batch_size=32,
#         epochs=50,
#         validation_data=(x_val, y_val),
#         verbose=1)